In [1]:
import os
import random
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
# import CountVectorizer

In [2]:
!pip install gdown

In [3]:
!gdown 1W-GGNPVxHD4bEgEZSLVMHOTpfrVy2o60

Downloading...
From: https://drive.google.com/uc?id=1W-GGNPVxHD4bEgEZSLVMHOTpfrVy2o60
To: /kaggle/working/input_paragraphs.csv
100%|██████████████████████████████████████| 12.0M/12.0M [00:00<00:00, 90.0MB/s]


In [4]:
#Concating all paragraphs in a theme, before vectorizing them. W Rizz
input_paragraphs = pd.read_csv('input_paragraphs.csv')
input_paragraphs.head(6)

,id,paragraph,theme
0,1,The 2008 Summer Olympics torch relay was run f...,2008_Summer_Olympics_torch_relay
1,2,After being lit at the birthplace of the Olymp...,2008_Summer_Olympics_torch_relay
2,3,In many cities along the North American and Eu...,2008_Summer_Olympics_torch_relay
3,4,The attacks on the torch in London and Paris w...,2008_Summer_Olympics_torch_relay
4,5,Prompted by the chaotic torch relays in Wester...,2008_Summer_Olympics_torch_relay
5,6,"In June 2008, the Beijing Games' Organizing Co...",2008_Summer_Olympics_torch_relay


In [5]:
theme_wise_paras  = {
    theme : list(input_paragraphs[input_paragraphs["theme"] == theme]["paragraph"]) for theme in input_paragraphs["theme"].unique()
}

In [6]:
# "".join([i for i in theme_wise_paras['ASCII']])
concated = {
    theme : "".join([i for i in theme_wise_paras[theme]]) for theme in theme_wise_paras
}

In [7]:
corpus = [[concated[i] for i in concated], [i for i in concated]] #for easy mapping, don't sue me

In [8]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True,  stop_words = "english", ngram_range=(1,3))
# just send in all your docs here
fitted_vectorizer=tfidf_vectorizer.fit(corpus[0])
tfidf_vectorizer_vectors=fitted_vectorizer.transform(corpus[0])

In [9]:
themess = corpus[1]
feature_names = fitted_vectorizer.get_feature_names() 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
vectorized = {
    themess[i] :  tfidf_vectorizer_vectors[i] for i in range(len(themess))
}

In [11]:
vectorized_df = {
    i :  pd.DataFrame(vectorized[i].T.todense(), index=feature_names, columns=["tfidf"]).reset_index() for i in vectorized 
}


In [12]:
n = 20

In [13]:
key_words = {
    i : vectorized_df[i].sort_values(by = ["tfidf"], ascending = False).index[:n] for i in vectorized_df
}

In [14]:
common_words_by_theme = {
    i : {
        j : len(key_words[j] & key_words[i]) for j in key_words
    } for i in key_words
}

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
def find_similar_themes(theme, threshold = 2, corpus = common_words_by_theme):
    sim_theme = []
    for i in corpus[theme]:
#         print(corpus[theme][i])
        if corpus[theme][i] >= threshold:
            sim_theme.append((i, corpus[theme][i]))
    return sim_theme


In [16]:

for k in range(2,6):
    n_similarity = []
    for i in themess:
        sim_themes = find_similar_themes(i, k)
#         print("{} {}".format(i, len(sim_themes)))
        n_similarity.append(len(sim_themes) - 1)
    #     print(sim_themes)
    print("Average number of \'similar\' themes {} at threshold: {}".format(sum(n_similarity)/len(n_similarity), k))

Average number of 'similar' themes 5.750692520775623 at threshold: 2
Average number of 'similar' themes 2.038781163434903 at threshold: 3
Average number of 'similar' themes 0.8642659279778393 at threshold: 4
Average number of 'similar' themes 0.38227146814404434 at threshold: 5
